In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
import numpy as np
import csv
import unidecode 
import pandas.core.algorithms as algos
from scipy.stats import kendalltau   
from funcoes_uteis import *
from dateutil.relativedelta import relativedelta

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.cross_validation import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import string

D:\Users\felipe.santos\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def diff_month(d1, d2):
    return (d1.year - d2.year)*12 + d1.month - d2.month

def periodicidade(x):
    if x == 'Trienal':
        return 36    
    elif x == 'Anual':
        return 12
    elif x == 'Semestral':
        return 6
    elif x == 'Trimestral':
        return 3
    else: 
        return 1

def marca_base(Perc, x):
    if x >= Perc[(len(Perc)-1)]:
        return len(Perc) +1
    else:
        for i in range(len(Perc)):
            if x < Perc[i]:
                return i + 1

def cria_curva(percentiles, variavel):
    Perc = list()
    for i in range(len(percentiles)):
        Perc.append(np.percentile(variavel, percentiles[i]))
    return Perc    

def contagens_anteriores(um_df):
    um_df['contagens_anteriores'] = range(len(um_df))
    return um_df

In [3]:
def cria_chave(df_recomendacao):
    if len(df_recomendacao.columns) == 3:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto2
        return df_recomendacao
    elif len(df_recomendacao.columns) == 4:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1+df_recomendacao.Produto2
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto3
        return df_recomendacao
    elif len(df_recomendacao.columns) == 5:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1+df_recomendacao.Produto2+df_recomendacao.Produto3    
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto4
        return df_recomendacao
    elif len(df_recomendacao.columns) == 6:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1+df_recomendacao.Produto2+df_recomendacao.Produto3+df_recomendacao.Produto4
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto5
        return df_recomendacao
    elif len(df_recomendacao.columns) == 7:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1+df_recomendacao.Produto2+df_recomendacao.Produto3+df_recomendacao.Produto4+df_recomendacao.Produto5
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto6     
        return df_recomendacao
    elif len(df_recomendacao.columns) == 8:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1+df_recomendacao.Produto2+df_recomendacao.Produto3+df_recomendacao.Produto4+df_recomendacao.Produto5+df_recomendacao.Produto6
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto7       
        return df_recomendacao 
    elif len(df_recomendacao.columns) == 9:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1+df_recomendacao.Produto2+df_recomendacao.Produto3+df_recomendacao.Produto4+df_recomendacao.Produto5+df_recomendacao.Produto6+df_recomendacao.Produto7
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto8       
        return df_recomendacao 
    elif len(df_recomendacao.columns) == 10:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1+df_recomendacao.Produto2+df_recomendacao.Produto3+df_recomendacao.Produto4+df_recomendacao.Produto5+df_recomendacao.Produto6+df_recomendacao.Produto7+df_recomendacao.Produto8
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto9       
        return df_recomendacao 
    elif len(df_recomendacao.columns) == 11:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1+df_recomendacao.Produto2+df_recomendacao.Produto3+df_recomendacao.Produto4+df_recomendacao.Produto5+df_recomendacao.Produto6+df_recomendacao.Produto7+df_recomendacao.Produto8+df_recomendacao.Produto9
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto10       
        return df_recomendacao 
    elif len(df_recomendacao.columns) == 12:
        df_recomendacao['chave_recomendacao'] = df_recomendacao.Produto1+df_recomendacao.Produto2+df_recomendacao.Produto3+df_recomendacao.Produto4+df_recomendacao.Produto5+df_recomendacao.Produto6+df_recomendacao.Produto7+df_recomendacao.Produto8+df_recomendacao.Produto9+df_recomendacao.Produto10
        df_recomendacao['ultimo_produto'] = df_recomendacao.Produto11        
        return df_recomendacao 

In [4]:
def Recomendacao_Base1(fim_janela_feature, janela_booking, df_base):
    aux_janela_feature = fim_janela_feature+ relativedelta(months=-12)
    inicio_janela_booking = fim_janela_feature
    fim_janela_booking = fim_janela_feature+ relativedelta(months=janela_booking)

    df_booking = df_base[(df_base.Instalacao_AnoMes >= inicio_janela_booking) & 
                         (df_base.Instalacao_AnoMes < fim_janela_booking)].copy()
    df_booking = df_booking[~df_booking.Servico.isnull()].copy()
    lista = list(df_booking.Servico.unique())
    lista = ['Comprou_' + str(i) for i in lista]
    dict_lista = {str(i): 'sum' for i in lista}
    df_booking.rename(columns= {'Servico': 'Comprou'}, inplace= True)
    ohe = ['Comprou']
    colunas = ['cd_ChaveCliente', 'Comprou']
    df_ohe_Servico = pd.get_dummies(df_booking[colunas], columns = ohe,)
    df_ohe_Servico = df_ohe_Servico.groupby('cd_ChaveCliente').agg(dict_lista)
    return df_ohe_Servico   

In [5]:
def calcula_recomendacao(df_recomendacao, chave):
    dict_lista = {'cd_ChaveCliente' : 'sum',}
    colunas_aux = ['ultimo_produto', 'chave_recomendacao']
    df_recomendacao1 = df_recomendacao.reset_index().groupby(colunas_aux).agg(dict_lista)
    df_recomendacao1.sort_values(['cd_ChaveCliente'], ascending= [0] ,inplace=True)
    df_recomendacao1.reset_index(inplace= True)
    
    df_recomendacao2 = df_recomendacao1[df_recomendacao1.chave_recomendacao != chave].copy()
    df_recomendacao2['chave_recomendacao'] = chave
    dict_lista = {'cd_ChaveCliente' : 'sum',}
    df_recomendacao2 = df_recomendacao2.reset_index().groupby(colunas_aux).agg(dict_lista)
    df_recomendacao2.sort_values(['cd_ChaveCliente'], ascending= [0] ,inplace=True)
    df_recomendacao2.reset_index(inplace= True)
    df_recomendacao2['perc_complementar'] = df_recomendacao2.cd_ChaveCliente*100/df_recomendacao2.cd_ChaveCliente.sum()
    
    df_recomendacao1 = df_recomendacao1[df_recomendacao1.chave_recomendacao == chave].copy()
    df_recomendacao1['perc'] = df_recomendacao1.cd_ChaveCliente*100/df_recomendacao1.cd_ChaveCliente.sum()
    
    aux = ['ultimo_produto','chave_recomendacao','perc_complementar']
    df_recomendacao_aux = pd.merge(df_recomendacao1, df_recomendacao2[aux], on=['chave_recomendacao', 'ultimo_produto'])
    df_recomendacao_aux['ratio_recomenda'] = df_recomendacao_aux.perc/df_recomendacao_aux.perc_complementar
    
    aux = ['ratio_recomenda','perc']
        
    RatioCorrigido = df_recomendacao_aux[df_recomendacao_aux.ratio_recomenda >= 1].copy()
    RatioCorrigido.sort_values(['ratio_recomenda'], ascending= [0] ,inplace=True)
        
    Freqencia = df_recomendacao_aux[df_recomendacao_aux.ratio_recomenda < 1].copy()
    Freqencia.sort_values(['perc'], ascending= [0] ,inplace=True)
        
    resultado = RatioCorrigido.append(Freqencia)
    resultado.reset_index(inplace= True, drop= True)
    resultado.reset_index(inplace= True)
    
    dict_lista = {'cd_ChaveCliente' : 'sum',}
    df_preenche_vazios = df_recomendacao.reset_index().groupby('ultimo_produto').agg(dict_lista)
    df_preenche_vazios.sort_values(['cd_ChaveCliente'], ascending= [0] ,inplace=True)
    df_preenche_vazios.reset_index(inplace= True)
    lista1 = list(df_preenche_vazios.ultimo_produto.unique())
    lista2 = list(resultado.ultimo_produto.unique())
    aux = list(set(lista1) - set(lista2))
    df_preenche_vazios = df_preenche_vazios[df_preenche_vazios.ultimo_produto.isin(aux)].copy()
    df_preenche_vazios['chave_recomendacao'] = chave
    
    resultado.drop('index', axis= 1, inplace= True)
    resultado = resultado.append(df_preenche_vazios)
    resultado.reset_index(inplace= True, drop= True)
    resultado.reset_index(inplace= True)
    
    resultado = resultado.pivot(index='chave_recomendacao', columns='index', values='ultimo_produto')
    return resultado

In [6]:
def junta_recomendacao(df_base, nivel):
    dict_lista = {'cd_ChaveCliente' : 'count',}
    c = nivel #começa no 2 e vai ate 10
    df_recomendacao = df_base.reset_index().groupby(colunas[:c]).agg(dict_lista)
    df_recomendacao.sort_values(['cd_ChaveCliente'], ascending= [0] ,inplace=True)
    df_recomendacao.reset_index(inplace= True)
    
    df_recomendacao = cria_chave(df_recomendacao)
    df_recomendacao = df_recomendacao[df_recomendacao.ultimo_produto != 'fim']
    listaChaves = list(df_recomendacao.chave_recomendacao.unique())
    
    result = calcula_recomendacao(df_recomendacao, listaChaves[0])
    for i in range(1,len(listaChaves)):
        result_aux = calcula_recomendacao(df_recomendacao, listaChaves[i])
        result = pd.concat([result, result_aux])
    return result

In [7]:
df_base = pd.read_csv('./Churn_Consumo_Recomendacao-23.10.2017.csv'
                      , error_bad_lines = False
                      , sep=';'
                      , dtype= {7: str}
                      , encoding='latin-1'
                      , header = 0) 

b'Skipping line 228640: expected 16 fields, saw 22\n'


In [8]:
df_base['Status'] = ['ativo' if s in ['Ativo', 'Atendido', 'Em ativação',
                                      'Aguardando ativação'] else 'inativo'
                     for s in df_base.Status]
df_base['nr_PrecoMensal'] = [x.replace(',', '.') for x in df_base.nr_PrecoMensal]
df_base['nr_PrecoMensal'] = df_base.nr_PrecoMensal.astype(float)
df_base = df_base[(df_base.nr_PrecoMensal > 0) & (df_base.Status == 'ativo')].copy()
df_base.drop_duplicates(['Provisioning'], keep='last', inplace= True)

In [9]:
col_datas = ['Data_Desativacao', 'Data_Fim', 'Instalacao', 'dt_Reativacao', 'Primeiro_Servico_LW']
converte_datetime(df_base, col_datas)
df_base.sort_values(['Instalacao'], ascending= 1 ,inplace=True)

In [10]:
df_base = df_base[df_base.Servico != 'Descontinuados'].copy()
df_base = df_base[df_base.Servico != 'Parcerias'].copy()
df_base = df_base[df_base.Servico != 'One Drive'].copy()
df_base = df_base[df_base.Servico != 'STREAMING_AUDIO_VIDEO'].copy()
df_base = df_base[df_base.Servico != 'Orago'].copy()
df_base = df_base[df_base.Servico != 'WR'].copy()

In [11]:
df_base = df_base.groupby('cd_ChaveCliente').apply(contagens_anteriores)

In [12]:
df_base.drop_duplicates(['Provisioning'], keep='last', inplace= True)
#df_base.sort_values(['Instalacao'], ascending= 1 ,inplace=True)
colunas = ['cd_ChaveCliente', 'Provisioning', 'Servico', 'contagens_anteriores']
df_base = df_base[colunas][df_base.contagens_anteriores <= 10].copy()
df_base['Servico'] = ['PABX Virtual' if s == 'PABX Virtual ' else s for s in df_base.Servico]

In [13]:
servico = [set(df_base.Servico)]
dict_lista = {'contagens_anteriores' : 'max',}
df_aux_ContagensAnteriores = df_base.groupby('cd_ChaveCliente').agg(dict_lista)
df_aux_ContagensAnteriores = df_aux_ContagensAnteriores.contagens_anteriores + 2
df_base = df_base.pivot(index='cd_ChaveCliente', columns='contagens_anteriores', values='Servico').copy()
colunas = ['Produto1','Produto2','Produto3','Produto4','Produto5','Produto6','Produto7','Produto8','Produto9','Produto10','Produto11']
df_base.columns = colunas
df_base.sort_values(colunas, ascending= [1,1,1,1,1,1,1,1,1,1,1] ,inplace=True)
df_base = pd.concat([df_aux_ContagensAnteriores, df_base], axis=1)    
df_base.reset_index(inplace= True)
df_base.rename(columns= {'index': 'cd_ChaveCliente'}, inplace= True)
df_base.fillna('fim', inplace = True)
df_base['chave_recomendacao'] = df_base.Produto1+df_base.Produto2+df_base.Produto3+df_base.Produto4+df_base.Produto5+df_base.Produto6+df_base.Produto7+df_base.Produto8+df_base.Produto9+df_base.Produto10
df_base['chave_recomendacao'] = [chave.replace('fim','') for chave in df_base.chave_recomendacao]

In [14]:
colunas = ['Produto1','Produto2','Produto3','Produto4','Produto5','Produto6','Produto7','Produto8','Produto9','Produto10','Produto11']

In [15]:
result2 = junta_recomendacao(df_base, 2)
result3 = junta_recomendacao(df_base, 3)
result4 = junta_recomendacao(df_base, 4)
result5 = junta_recomendacao(df_base, 5)
result6 = junta_recomendacao(df_base, 6)
result7 = junta_recomendacao(df_base, 7)
result8 = junta_recomendacao(df_base, 8)
result9 = junta_recomendacao(df_base, 9)
result10 = junta_recomendacao(df_base, 10)

In [16]:
frames = [result2,result3,result4,result5,result6,result7,result8,result9,result10]
df_MatrizRecomendacao = pd.concat(frames)

In [17]:
colunas = []
for i in range(len(df_MatrizRecomendacao.columns)):
    colunas.append('Recomendacao'+str(i+1))

In [18]:
df_MatrizRecomendacao.columns = colunas

In [19]:
df_MatrizRecomendacao.to_pickle('df_MatrizRecomendacao_24102017.pkl')

In [7]:
df_MatrizRecomendacao = pd.read_pickle('./df_MatrizRecomendacao_23102017.pkl')

In [14]:
df_MatrizRecomendacao.Recomendacao5.value_counts()

Email                   6211
Exchange                5188
Cloud Server Pro        3374
Hospedagem               215
Registro Dominio         182
Email MKT                 49
SSL                       25
SMTP                      22
Servidor Dedicado         20
Revenda Hosp.             13
GestÆo Personalizada      10
Criador de Sites           9
Licencas - Dedicado        9
WR                         8
PABX Virtual               5
Cloud VPS                  5
Gateway Pagtos             2
Revenda EMKT               1
Windows Streaming          1
Google Apps                1
Licencas - Cloud           1
WebStore                   1
Name: Recomendacao5, dtype: int64

In [20]:
len(df_MatrizRecomendacao.columns)

27

In [22]:
df_MatrizRecomendacao.head(10).T

chave_recomendacao,Hospedagem,Registro Dominio,Criador de Sites,Email,Email MKT,Cloud Server Pro,Exchange,WR,Revenda Hosp.,PABX Virtual
Recomendacao1,SSL,Criador de Sites,Registro Dominio,Email,Windows Streaming,Licencas - Dedicado,Exchange,Revenda EMKT,Revenda EMKT,PABX Virtual
Recomendacao2,Email MKT,Hospedagem,Hospedagem,Backup de Sites,WR,GestÆo Personalizada,Office 365,Revenda Hosp.,Revenda Hosp.,Revenda EMKT
Recomendacao3,SMTP,Google Apps,Criador de Sites,SMTP,WebDesk,Licencas - Cloud,WebDesk,SMTP,Backup de Sites,WebStore
Recomendacao4,Object Storage,WR,Email MKT,Object Storage,Revenda EMKT,Cloud Server Pro,Servidor Dedicado,WR,Office 365,Cloud VPS
Recomendacao5,Gateway Pagtos,Email,Email,WebStore,Servidor Dedicado,Servidor Dedicado,Registro Dominio,Email MKT,Servidor Dedicado,Revenda Hosp.
Recomendacao6,Windows Streaming,Registro Dominio,PABX Virtual,Registro Dominio,SMTP,Object Storage,Hospedagem,SSL,Cloud VPS,Servidor Dedicado
Recomendacao7,WebStore,Exchange,Exchange,Servidor Dedicado,Revenda Hosp.,WebDesk,Email MKT,WebDesk,SMTP,Google Apps
Recomendacao8,Backup de Sites,Revenda Hosp.,Revenda Hosp.,Email MKT,GestÆo Personalizada,Revenda Hosp.,Criador de Sites,Gateway Pagtos,WR,Hospedagem
Recomendacao9,Registro Dominio,Email MKT,SSL,Hospedagem,Hospedagem,Hospedagem,Email,Cloud Server Pro,PABX Virtual,Registro Dominio
Recomendacao10,Office 365,Cloud Server Pro,SMTP,Criador de Sites,WebStore,Registro Dominio,PABX Virtual,Registro Dominio,WebDesk,Email MKT


In [23]:
df_base = pd.read_csv('./Churn_Consumo_Recomendacao-23.10.2017.csv'
                      , error_bad_lines = False
                      , sep=';'
                      , dtype= {7: str}
                      , encoding='latin-1'
                      , header = 0) 

b'Skipping line 228640: expected 16 fields, saw 22\n'


In [24]:
df_base['Status'] = ['ativo' if s in ['Ativo', 'Atendido', 'Em ativação',
                                      'Aguardando ativação'] else 'inativo'
                     for s in df_base.Status]
df_base['nr_PrecoMensal'] = [x.replace(',', '.') for x in df_base.nr_PrecoMensal]
df_base['nr_PrecoMensal'] = df_base.nr_PrecoMensal.astype(float)
df_base = df_base[(df_base.nr_PrecoMensal > 0) & (df_base.Status == 'ativo')].copy()
df_base.drop_duplicates(['Provisioning'], keep='last', inplace= True)

In [25]:
col_datas = ['Data_Desativacao', 'Data_Fim', 'Instalacao', 'dt_Reativacao', 'Primeiro_Servico_LW']
converte_datetime(df_base, col_datas)
df_base.sort_values(['Instalacao'], ascending= 1 ,inplace=True)

In [26]:
df_base = df_base[df_base.Servico != 'Descontinuados'].copy()
df_base = df_base[df_base.Servico != 'Parcerias'].copy()
df_base = df_base[df_base.Servico != 'One Drive'].copy()
df_base = df_base[df_base.Servico != 'STREAMING_AUDIO_VIDEO'].copy()
df_base = df_base[df_base.Servico != 'Orago'].copy()
df_base = df_base[df_base.Servico != 'WR'].copy()

In [27]:
df_base = df_base.groupby('cd_ChaveCliente').apply(contagens_anteriores)

In [28]:
df_base.drop_duplicates(['Provisioning'], keep='last', inplace= True)
df_base.sort_values(['Instalacao'], ascending= 1 ,inplace=True)
colunas = ['cd_ChaveCliente', 'Provisioning', 'Servico', 'contagens_anteriores', 'Instalacao']
df_base = df_base[colunas][df_base.contagens_anteriores <= 10].copy()
df_base['Servico'] = ['PABX Virtual' if s == 'PABX Virtual ' else s for s in df_base.Servico]

In [47]:
data = df_base.Instalacao.max()

datetime.datetime(2017, 10, 22, 0, 0)

In [30]:
dict_lista = {'contagens_anteriores' : 'max',}
df_aux_ContagensAnteriores = df_base.groupby('cd_ChaveCliente').agg(dict_lista)
df_aux_ContagensAnteriores = df_aux_ContagensAnteriores.contagens_anteriores + 2
df_base = df_base.pivot(index='cd_ChaveCliente', columns='contagens_anteriores', values='Servico').copy()
colunas = ['Produto1','Produto2','Produto3','Produto4','Produto5','Produto6','Produto7','Produto8','Produto9','Produto10','Produto11']
df_base.columns = colunas
df_base.sort_values(colunas, ascending= [1,1,1,1,1,1,1,1,1,1,1] ,inplace=True)
df_base = pd.concat([df_aux_ContagensAnteriores, df_base], axis=1)    
df_base.reset_index(inplace= True)
df_base.rename(columns= {'index': 'cd_ChaveCliente'}, inplace= True)
df_base.fillna('fim', inplace = True)
df_base['chave_recomendacao'] = df_base.Produto1+df_base.Produto2+df_base.Produto3+df_base.Produto4+df_base.Produto5+df_base.Produto6+df_base.Produto7+df_base.Produto8+df_base.Produto9+df_base.Produto10
df_base['chave_recomendacao'] = [chave.replace('fim','') for chave in df_base.chave_recomendacao]

In [31]:
df_base = pd.merge(df_base, df_aux_ContagensAnteriores.reset_index(), how='inner', on=['cd_ChaveCliente'])

In [32]:
Recomendacao = []
for i in range(len(df_MatrizRecomendacao.columns)):
    Recomendacao.append('Recomendacao'+str(i+1))
df_base = pd.merge(df_base, df_MatrizRecomendacao[Recomendacao].reset_index(), how='inner', on=['chave_recomendacao'])

In [38]:
df_base.shape

(265917, 42)

In [42]:
colunas = Recomendacao
colunas.append('chave_recomendacao')
colunas.append('cd_ChaveCliente')

In [45]:
df_base[colunas].sample()

,Recomendacao1,Recomendacao2,Recomendacao3,Recomendacao4,Recomendacao5,Recomendacao6,Recomendacao7,Recomendacao8,Recomendacao9,Recomendacao10,...,Recomendacao20,Recomendacao21,Recomendacao22,Recomendacao23,Recomendacao24,Recomendacao25,Recomendacao26,Recomendacao27,chave_recomendacao,cd_ChaveCliente
219956,Google Apps,Registro Dominio,Criador de Sites,Hospedagem,Email MKT,Email,SSL,SMTP,Cloud Server Pro,Exchange,...,Windows Streaming,Revenda EMKT,WebDesk,WebStore,Backup de Sites,Object Storage,Office 365,NaN,HospedagemRegistro DominioRegistro Dominio,__08936349000141__


In [ ]:
NomeCSV = '/home/felipe/Algoritmos_todosProdutos/Churn/Recomendacao_'+str(df_base.Instalacao.max().year)+'_'+str(df_base.Instalacao.max().month)+'_'+str(df_base.Instalacao.max().day)+'.csv'
df_base[colunas].to_csv(NomeCSV)

#  

#  

In [255]:
for i in range(10):
    df_base['AcertoRecomendacao'+str(i+1)] = 0
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou1'],df_base['Recomendacao'+str(i+1)])]
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou2'],df_base['Recomendacao'+str(i+1)])]+df_base['AcertoRecomendacao'+str(i+1)]
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou3'],df_base['Recomendacao'+str(i+1)])]+df_base['AcertoRecomendacao'+str(i+1)]
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou4'],df_base['Recomendacao'+str(i+1)])]+df_base['AcertoRecomendacao'+str(i+1)]
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou5'],df_base['Recomendacao'+str(i+1)])]+df_base['AcertoRecomendacao'+str(i+1)]
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou6'],df_base['Recomendacao'+str(i+1)])]+df_base['AcertoRecomendacao'+str(i+1)]
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou7'],df_base['Recomendacao'+str(i+1)])]+df_base['AcertoRecomendacao'+str(i+1)]
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou8'],df_base['Recomendacao'+str(i+1)])]+df_base['AcertoRecomendacao'+str(i+1)]
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou9'],df_base['Recomendacao'+str(i+1)])]+df_base['AcertoRecomendacao'+str(i+1)]
    df_base['AcertoRecomendacao'+str(i+1)] = [1 if r == c else 0 for c, r in zip(df_base['Comprou10'],df_base['Recomendacao'+str(i+1)])]+df_base['AcertoRecomendacao'+str(i+1)]

In [268]:
tot = df_base.qtd_compras.sum()
r1 = df_base.AcertoRecomendacao1.sum()
r2 = df_base.AcertoRecomendacao2.sum()
r3 = df_base.AcertoRecomendacao3.sum()
r4 = df_base.AcertoRecomendacao4.sum()
r5 = df_base.AcertoRecomendacao5.sum()
r6 = df_base.AcertoRecomendacao6.sum()
r7 = df_base.AcertoRecomendacao7.sum()
r8 = df_base.AcertoRecomendacao8.sum()
r9 = df_base.AcertoRecomendacao9.sum()
r10 = df_base.AcertoRecomendacao10.sum()


In [274]:
1/30

0.03333333333333333

In [273]:
print(tot)
print((r1)/tot)
print((r1+r2)/tot)
print((r1+r2+r3)/tot)
print((r1+r2+r3+r4)/tot)
print((r1+r2+r3+r4+r5)/tot)
print((r1+r2+r3+r4+r5+r6)/tot)
print((r1+r2+r3+r4+r5+r6+r7)/tot)
print((r1+r2+r3+r4+r5+r6+r7+r8)/tot)
print((r1+r2+r3+r4+r5+r6+r7+r8+r9)/tot)
print((r1+r2+r3+r4+r5+r6+r7+r8+r9+r10)/tot)

23461
0.278334256852
0.436128042283
0.51587741358
0.609010698606
0.671028515409
0.723541196027
0.753676313883
0.834704403052
0.858403307617
0.87528238353


In [269]:
print(tot)
print(r1/tot)
print(r2/tot)
print(r3/tot)
print(r4/tot)
print(r5/tot)
print(r6/tot)
print(r7/tot)
print(r8/tot)
print(r9/tot)
print(r10/tot)

23461
0.278334256852
0.157793785431
0.079749371297
0.0931332850262
0.0620178168024
0.0525126806189
0.0301351178552
0.0810280891693
0.023698904565
0.0168790759132


In [256]:
df_base.sample(3).T

,12844,9459,14125
cd_ChaveCliente,__07737128000181__,__21143116000150__,__18568267000226__
chave_recomendacao,PABX VirtualPABX VirtualPABX VirtualPABX Virtu...,Registro DominioCriador de Sites,ExchangeExchangeHospedagemExchangeExchange
Comprou1,Hospedagem,Criador de Sites,Email MKT
Comprou2,Hospedagem,Registro Dominio,None
Comprou3,None,None,None
Comprou4,None,None,None
Comprou5,None,None,None
Comprou6,None,None,None
Comprou7,None,None,None
Comprou8,None,None,None


In [124]:
colunas_recomendacao = (df_MatrizRecomendacao.columns)

In [122]:
colunas[0]

'Comprou1'

In [ ]:
df_base['acertos1'] = []

In [126]:
df_base.sample(3).T

,12210,2714,2581
cd_ChaveCliente,__05405249726__,__20188572821__,__07725847000182__
contagens_anteriores,8,3,3
Produto1,Registro Dominio,Hospedagem,Hospedagem
Produto2,Registro Dominio,Hospedagem,Hospedagem
Produto3,Registro Dominio,fim,fim
Produto4,Registro Dominio,fim,fim
Produto5,Registro Dominio,fim,fim
Produto6,Registro Dominio,fim,fim
Produto7,Registro Dominio,fim,fim
Produto8,fim,fim,fim


# Código Auxiliar

In [275]:
################################################
################################################
######Codigo Auxiliar ##########################
################################################


chave = listaChaves[1]
dict_lista = {'cd_ChaveCliente' : 'sum',}
colunas_aux = ['ultimo_produto', 'chave_recomendacao']
df_recomendacao1 = df_recomendacao.reset_index().groupby(colunas_aux).agg(dict_lista)
df_recomendacao1.sort_values(['cd_ChaveCliente'], ascending= [0] ,inplace=True)
df_recomendacao1.reset_index(inplace= True)

df_recomendacao2 = df_recomendacao1[df_recomendacao1.chave_recomendacao != chave].copy()
df_recomendacao2['chave_recomendacao'] = chave
dict_lista = {'cd_ChaveCliente' : 'sum',}
df_recomendacao2 = df_recomendacao2.reset_index().groupby(colunas_aux).agg(dict_lista)
df_recomendacao2.sort_values(['cd_ChaveCliente'], ascending= [0] ,inplace=True)
df_recomendacao2.reset_index(inplace= True)
df_recomendacao2['perc_complementar'] = df_recomendacao2.cd_ChaveCliente*100/df_recomendacao2.cd_ChaveCliente.sum()

df_recomendacao1 = df_recomendacao1[df_recomendacao1.chave_recomendacao == chave].copy()
df_recomendacao1['perc'] = df_recomendacao1.cd_ChaveCliente*100/df_recomendacao1.cd_ChaveCliente.sum()

aux = ['ultimo_produto','chave_recomendacao','perc_complementar']
df_recomendacao_aux = pd.merge(df_recomendacao1, df_recomendacao2[aux], on=['chave_recomendacao', 'ultimo_produto'])
df_recomendacao_aux['ratio_recomenda'] = df_recomendacao_aux.perc/df_recomendacao_aux.perc_complementar

aux = ['ratio_recomenda','perc']
    
RatioCorrigido = df_recomendacao_aux[df_recomendacao_aux.ratio_recomenda >= 1].copy()
RatioCorrigido.sort_values(['ratio_recomenda'], ascending= [0] ,inplace=True)
    
Freqencia = df_recomendacao_aux[df_recomendacao_aux.ratio_recomenda < 1].copy()
Freqencia.sort_values(['perc'], ascending= [0] ,inplace=True)
    
resultado = RatioCorrigido.append(Freqencia)
resultado.reset_index(inplace= True, drop= True)
resultado.reset_index(inplace= True)

dict_lista = {'cd_ChaveCliente' : 'sum',}
df_preenche_vazios = df_recomendacao.reset_index().groupby('ultimo_produto').agg(dict_lista)
df_preenche_vazios.sort_values(['cd_ChaveCliente'], ascending= [0] ,inplace=True)
df_preenche_vazios.reset_index(inplace= True)
lista1 = list(df_preenche_vazios.ultimo_produto.unique())
lista2 = list(resultado.ultimo_produto.unique())
aux = list(set(lista1) - set(lista2))
df_preenche_vazios = df_preenche_vazios[df_preenche_vazios.ultimo_produto.isin(aux)].copy()
df_preenche_vazios['chave_recomendacao'] = chave

resultado.drop('index', axis= 1, inplace= True)
resultado = resultado.append(df_preenche_vazios)
resultado.reset_index(inplace= True, drop= True)
resultado.reset_index(inplace= True)

resultado = resultado.pivot(index='chave_recomendacao', columns='index', values='ultimo_produto')

NameError: name 'listaChaves' is not defined

In [1483]:
resultado.T

chave_recomendacao,HospedagemRegistro Dominio
index,
0,Object Storage
1,Registro Dominio
2,Schedulista
3,SSL
4,Google Adwords
5,Criador de Sites
6,WebStore
7,Hospedagem
8,Email MKT


In [10]:
#Churn_Consumo_Recomendacao_18.08.2017
df_base = pd.read_csv('./Churn_Consumo_Recomendacao_dd.mm.yyyy.csv'
                      , error_bad_lines = False
                      , sep=';'
                      , dtype= {7: str}
                      , encoding='latin-1'
                      , header = 0)
df_base['Status'] = ['ativo' if s in ['Ativo', 'Atendido', 'Em ativação',
                                      'Aguardando ativação'] else 'inativo'
                     for s in df_base.Status]
df_base['fl_Dev'] = df_base['fl_Dev'].astype(float,)
df_base = df_base[(df_base.nr_PrecoMensal > '0,00')].copy()
df_base.sort_values(['Instalacao'], ascending= 1 ,inplace=True)
df_base.drop_duplicates(['Provisioning'], keep='last', inplace= True)
col_datas = ['Data_Desativacao', 'Data_Fim', 'Instalacao', 'dt_Reativacao', 'Primeiro_Servico_LW']
converte_datetime(df_base, col_datas)
df_base['Data_Fim_flag'] = [1 if d_fim != datetime(1900,1,1) else 0 for d_fim in df_base.Data_Fim]
df_base['Data_Desativacao_flag'] = [1 if f_des != datetime(1900,1,1) else 0 for f_des in df_base.Data_Desativacao]
df_base['Data_Desativacao_flag'] = [1 if f_des != datetime(1900,1,1) else 0 for f_des in df_base.Data_Desativacao]
df_base['Data_churn'] = df_base[['Data_Fim', 'Data_Desativacao']].min(axis = 1).astype('datetime64[ns]')
df_base['Data_churn'] = [d_fim if ((d_des < d_rea < d_fim) & f_fim & f_des) else d_chu
                              for d_des, d_rea, d_fim, f_fim, f_des, d_chu in 
                              zip(df_base.Data_Desativacao, df_base.dt_Reativacao, 
                                  df_base.Data_Fim, df_base.Data_Fim_flag, 
                                  df_base.Data_Desativacao_flag, df_base.Data_churn)]
df_base['Data_churn_flag'] = [1 if d > datetime(1900, 1, 1) else 0 for d in df_base.Data_churn]
#######################################
df_base = df_base[df_base.Primeiro_Servico_LW >= datetime(1990,1,1)].copy()  #
#######################################
df_base['Primeiro_Servico_LW_AnoMes'] = [datetime(d.year, d.month, 1) for d in df_base.Primeiro_Servico_LW]
df_base['Instalacao_AnoMes'] = [datetime(d.year, d.month, 1) for d in df_base.Instalacao]
df_base['Data_churn_AnoMes'] = [datetime(d.year, d.month, 1) for d in df_base.Data_churn]
df_base['fl_ServicoPai'] = [1 if s in ['SIM', 'Sim', 'sim'] else 0 for s in df_base.fl_ServicoPai]
df_base['fl_GerenteConta'] = [0 if s== 1 else 1 for s in df_base.id_GerenteConta]
df_base['nr_PrecoMensal'] = [x.replace(',', '.') for x in df_base.nr_PrecoMensal]
df_base['nr_PrecoMensal'] = df_base.nr_PrecoMensal.astype(float)
df_base['MesesParaChurn'] = [diff_month(ch, ins) if ch > datetime(1900, 1, 1) else 0
                           for ch, ins in  zip(df_base.Data_churn, df_base.Instalacao)]
df_base['FlagChurn'] = [1 if d!= 0 else 0 for d in df_base.MesesParaChurn]